In [ ]:
import atlite
import xarray as xr
import rasterio as rio
import pandas as pd
import numpy as np
import geopandas as gpd
import shapely
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
plt.style.use('dark_background')

from rasterio.plot import show
from atlite.gis import shape_availability, ExclusionContainer

from exp_utils import wgp_functions

plt.rcParams['figure.figsize'] = [8, 8]

###### Packages for turbine placement: 

from skimage import draw, morphology
from scipy.ndimage import binary_dilation as dilation
import math


### 1. Set-up of initial landuse availability matrix

In [ ]:
# Excel with necessary information on the shapes luisa / corine cover, as well as the gridcodes
gc_excel = pd.read_excel('./gridcode_dict.xlsx', sheet_name = 'py_dict_format', 
                         converters = {'in_gc_luisa' : int, 'in_gc_corine' : int, 'ex_gc_luisa' : int, 'ex_gc_corine': int})

# set up country shapes 
raster = rio.open('C:/CaT/Masterthesis/data/corine.tif')
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

shapes = wgp_functions.tif_bb_filter(raster = raster, shapes = world, output_crs = 'shapes')

# There are no shapes for Liechtenstein & Malta in the gpd library. Turkey and Iceland are excluded as they are not in focus of the analysis 
shapes = shapes[shapes['name'].isin(list(gc_excel['countries']))]
shapes.set_index('name', inplace=True)
shapes.drop(['Iceland', 'Turkey'], inplace = True)

shapes.plot()

# set up gridcode dictionary. see ./gridcodes.xlsx for further reference
gridcodes = {
    'inclusion' : 
    {
        'luisa' : dict(zip(gc_excel['in_gc_type'], gc_excel['in_gc_luisa'])),
        'corine' : dict(zip(gc_excel['in_gc_type'], gc_excel['in_gc_corine']))
    },
    'exclusion' :
    {
        'luisa' : dict(zip(gc_excel['ex_gc_type_luisa'].dropna(), gc_excel['ex_gc_luisa'].dropna())),
        'corine' : dict(zip(gc_excel['ex_gc_type_corine'].dropna(), gc_excel['ex_gc_corine'].dropna()))
    }
}


LUISA = 'C:/CaT/Masterthesis/data/luisa.tif'
CORINE = 'C:/CaT/Masterthesis/data/corine.tif'

In [ ]:
cntry = 'Luxembourg'

# set up raster
exc_corine = ExclusionContainer(res=100)
exc_corine.add_raster(CORINE, codes = list(gridcodes['inclusion']['corine'].values()), crs=3035, invert=True)
exc_corine.add_raster(CORINE, codes = list(gridcodes['exclusion']['corine'].values()), crs=3035,  buffer=1000)

# pick a country
country = shapes.loc[[cntry]].geometry.to_crs(exc_corine.crs)

# get availibility mask and transformation affine 
masked_c, transform_c = shape_availability(country, exc_corine)
eligible_share_c = masked_c.sum().astype(float) * exc_corine.res**2 / country.geometry.item().area

#Save mask as geotiff
#write_geotiff(f'./{cntry}.tif', mask = masked_c, trans = transform_c) 

# plot
fig, ax = plt.subplots()
ax = show(masked_c, transform=transform_c, cmap='Greens', ax=ax)
country.plot(ax=ax, edgecolor='k', color='None')
ax.set_title(f'CORINE Data: Eligible area (green) {eligible_share_c * 100:2.2f}%');

### 2. Initial elipse testing and manual one-iteration turbine placement

In [ ]:
# How to draw an elipsis
rotor = 1

img = np.zeros((100, 100), 'uint8')

rr, cc = draw.ellipse(50, 50, 20 , 10 , img.shape, rotation = 0)#math.pi/-1.1)
img[rr,cc] = 1

show(img)

In [ ]:
# Initial landuse availability matrix
masked_c.shape

In [ ]:
# Two further dimensions, one which will capture the turbine centers and one which captures the blocked area by the eilipse

turbine_av = ~np.zeros(shape = masked_c.shape, dtype = bool)
turbine_spot = np.zeros(shape = masked_c.shape, dtype = bool) 

mask = np.stack((masked_c, turbine_av, turbine_spot))

mask.shape

In [ ]:
fig, ax = plt.subplots(1,3, figsize = (12,8))

show(mask[0, :, :], vmin = 0, vmax = 1, cmap = "Greens", ax = ax[0])
show(mask[1, :, :], vmin = 0, vmax = 1, cmap = "Greens", ax = ax[1])
show(mask[2, :, :], vmin = 0, vmax = 1, cmap = "Greens", ax = ax[2])

ax[0].set_title("Landuse av.")
ax[1].set_title("Turbine area")
ax[2].set_title("Turbine center")

In [ ]:
# Find the first index from NW (upper left) that is eligible according to landuse availability
idx = np.unravel_index(np.argmax(mask[0, :, :], axis=None), mask[0, :, :].shape)
idx

In [ ]:
#Place a turbine
mask[2, idx[0] , idx[1]] = True

rotor = 180/50

img = ~np.zeros(mask[0, :, :].shape, bool)

rr, cc = draw.ellipse(idx[0], idx[1], rotor * 6, rotor * 12, img.shape, rotation = math.pi/-1.1)

mask[0, rr,cc] = False
mask[1, rr,cc] = False

In [ ]:
fig, ax = plt.subplots(1,3, figsize = (12,8))

show(mask[0, :, :], vmin = 0, vmax = 1, cmap = "Greens", ax = ax[0])
show(mask[1, :, :], vmin = 0, vmax = 1, cmap = "Greens", ax = ax[1])
show(dilation(mask[2, :, :], iterations = 5), vmin = 0, vmax = 1, cmap = "Greys", ax = ax[2])

ax[0].set_title("Landuse av.")
ax[1].set_title("Turbine area")
ax[2].set_title("Turbine center")

### 3. Automated sequential turbine placement

In [ ]:
%%time

turbine_av = ~np.zeros(shape = masked_c.shape, dtype = bool)
turbine_spot = np.zeros(shape = masked_c.shape, dtype = bool) 

mask = np.stack((masked_c, turbine_av, turbine_spot))

rotoer = 180/50

while mask[0, :, :].max() == 1:

    idx = np.unravel_index(np.argmax(mask[0, :, :], axis=None), mask[0, :, :].shape)

    mask[2, idx[0] , idx[1]] = True

    rr, cc = draw.ellipse(idx[0], idx[1], rotor * 6, rotor * 12, mask[0, :, :].shape , rotation = math.pi/4)

    mask[0, rr,cc] = False
    mask[1, rr,cc] = False
    
print(f'Successfully built {mask[2, :, :].sum()} turbines') 

In [ ]:
fig, ax = plt.subplots(1,3, figsize = (12,8))

show(mask[0, :, :], vmin = 0, vmax = 1, cmap = "Greens", ax = ax[0])
show(mask[1, :, :], vmin = 0, vmax = 1, cmap = "Greens", ax = ax[1])
show(dilation(mask[2, :, :], iterations = 5), vmin = 0, vmax = 1, cmap = "Greys", ax = ax[2])

ax[0].set_title("Landuse av.")
ax[1].set_title("Turbine occupied area")
ax[2].set_title("Turbine center")

In [ ]:
import matplotlib
c_transparent = matplotlib.colors.colorConverter.to_rgba('white',alpha = 0)
c_red= matplotlib.colors.colorConverter.to_rgba('black',alpha = 1)
cmap_tr = matplotlib.colors.LinearSegmentedColormap.from_list('rb_cmap',[c_transparent,c_red],512)

fig, ax = plt.subplots()
show(mask[1, :, :], vmin = 0, vmax = 1, cmap = "Greens", ax=ax)
show(dilation(mask[2, :, :], iterations = 5), vmin = 0, vmax = 1, cmap = cmap_tr, ax=ax)
ax.set_title(f"Turbine placements with respective buffer elipse (n={mask[2, :, :].sum()})")
#plt.savefig("3placement.png", bbox_inches = "tight", dpi = 200)

plt.show()

### Final function and incorporation of rotation input from wind direction

In [ ]:
#Function wrapper

def place_turbines(res, landuse_mask, wind_dir, rotor_diam, r_radius_factor, c_radius_factor):    
    
    total_av = landuse_mask
    
    if np.isscalar(wind_dir):
        wind_matrix = np.full(total_av.shape, wind_dir).astype(int)
    else:
        wind_matrix = wind_dir
    
    turbine_av = ~np.zeros(shape = total_av.shape, dtype = bool) 
    turbine_spot = np.zeros(shape = total_av.shape, dtype = bool) 
    
    mask = np.stack((total_av, turbine_av, turbine_spot))
    print(mask.dtype)
    
    rotor = rotor_diam / res  
    
    while mask[0, :, :].max() == 1:

        idx = np.unravel_index(np.argmax(mask[0, :, :], axis=None), mask[0, :, :].shape)

        # Save turbine placement center
        mask[2, idx[0] , idx[1]] = True

        # Draw elipse based on predominant wind direcition   
        rr, cc = draw.ellipse(idx[0], idx[1], rotor * r_radius_factor, rotor * c_radius_factor, mask[0, :, :].shape , 
                              rotation = math.pi/wind_matrix[idx[0], idx[1]])

        mask[0, rr,cc] = False
        mask[1, rr,cc] = False
    
    print(f'Successfully built {mask[2, :, :].sum()} turbines')
    
    return mask

In [ ]:
np.full(masked_c.shape, 4).astype(int)

In [ ]:
%%time

wind_dir = np.random.randint(-10, 10, masked_c.shape)
wind_dir[wind_dir == 0] = 1

turbine_mask = place_turbines(res = 50, 
                              landuse_mask = masked_c,
                              wind_dir =  wind_dir,
                              rotor_diam = 130,
                              r_radius_factor = 6,
                              c_radius_factor = 12) 

In [ ]:
fig, ax = plt.subplots()
show(turbine_mask[1, :, :], vmin = 0, vmax = 1, cmap = "Greens", ax=ax)
show(dilation(turbine_mask[2, :, :], iterations = 5), vmin = 0, vmax = 1, cmap = cmap_tr, ax=ax)
ax.set_title(f"Turbine placements with respective buffer elipse (n={turbine_mask[2, :, :].sum()})")

plt.show()

### 4. Computation of wind direction from u and v components of wind

In [ ]:
# https://confluence.ecmwf.int/pages/viewpage.action?pageId=133262398